In [16]:

!pip install pandas scikit-learn openpyxl joblib streamlit pyngrok


In [17]:

import pandas as pd
import random
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


In [18]:
places_info = [
    {
        "name": "สวนบิ๊กเต้",
        "category": ["ฟาร์ม", "ดอกไม้"],
        "district": "มวกเหล็ก",
        "description": "สวนเบญจมาศบิ๊กเต้ เป็นแหล่งท่องเที่ยวเชิงเกษตรที่เต็มไปด้วยดอกเบญจมาศหลากสี เหมาะสำหรับคนชอบถ่ายรูปกับทุ่งดอกไม้ และยังมีคาเฟ่เล็กๆ ให้นั่งพักผ่อน",
        "address": "213 หมู่ 5 อำเภอมวกเหล็ก จังหวัดสระบุรี",
        "open_time": "08:00-17:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "พ.ย.-ม.ค. ช่วงดอกเบญจมาศบาน",
        "location": "https://maps.app.goo.gl/U1DEvCSo5kux2h7w5"
    },
    {
        "name": "วัดพระพุทธบาท",
        "category": ["วัด"],
        "district": "พระพุทธบาท",
        "description": "วัดพระพุทธบาทราชวรมหาวิหาร เป็นวัดสำคัญคู่บ้านคู่เมืองที่ประดิษฐานรอยพระพุทธบาท ค้นพบตั้งแต่สมัยอยุธยา เป็นศูนย์รวมศรัทธาของชาวไทย",
        "address": "ตำบลขุนโขลน อำเภอพระพุทธบาท จังหวัดสระบุรี",
        "open_time": "08:00-17:00",
        "entry_fee": "ไม่มีค่าเข้า (บางเทศกาลอาจมีค่าใช้จ่ายเพิ่มเติม)",
        "best_time": "ก.พ. ช่วงงานเทศกาลนมัสการรอยพระพุทธบาท",
        "location": "https://maps.app.goo.gl/W7zJrMa53oRSsxSW9"
    },
    {
        "name": "อ่างเก็บน้ำมวกเหล็ก",
        "category": ["ธรรมชาติ"],
        "district": "วังม่วง",
        "description": "อ่างเก็บน้ำมวกเหล็ก เขื่อนดินที่ใช้ผลิตน้ำประปาและการเกษตร เป็นสถานที่พักผ่อน ปั่นจักรยาน และชมวิวภูเขาสวย",
        "address": "ตำบลคำพราน อำเภอวังม่วง จังหวัดสระบุรี",
        "open_time": "06:00-16:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี โดยเฉพาะฤดูหนาว บรรยากาศดี",
        "location": "https://maps.app.goo.gl/igwfGEQcKVtoJueS7"
    },
    {
        "name": "ทุ่งสิริสมัย",
        "category": ["ธรรมชาติ", "ดอกไม้"],
        "district": "แก่งคอย",
        "description": "ทุ่งดอกคอสมอสสีชมพูอลังการ บานสะพรั่งในฤดูหนาว เหมาะสำหรับการถ่ายรูป",
        "address": "ตำบลสองคอน อำเภอแก่งคอย จังหวัดสระบุรี",
        "open_time": "ธ.ค.-ม.ค.",
        "entry_fee": "เข้าชมฟรี",
        "best_time": "ธ.ค.-ม.ค. ช่วงดอกคอสมอสบาน",
        "location": "https://maps.app.goo.gl/tXAtB6jJPyQUFHgU6"
    },
    {
        "name": "ตลาดหัวปลี",
        "category": ["ตลาด"],
        "district": "เฉลิมพระเกียรติ",
        "description": "ตลาดเกษตรชุมชน สินค้าเกษตรสดใหม่ ของดี OTOP ของอร่อยราคาย่อมเยา",
        "address": "ทางหลวงหมายเลข 21 ตำบลพุแค อำเภอเฉลิมพระเกียรติ",
        "open_time": "09:00-17:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "เช้า-บ่าย",
        "location": "https://maps.app.goo.gl/3TqU2JSUDYM89bTY6"
    },
    {
        "name": "สวนพฤกษศาสตร์พุแค",
        "category": ["ธรรมชาติ"],
        "district": "เฉลิมพระเกียรติ",
        "description": "สวนพฤกษศาสตร์พุแค แหล่งเรียนรู้และอนุรักษ์พันธุ์ไม้หลากชนิด เหมาะกับครอบครัวและนักเรียน",
        "address": "123 หมู่ 1 ตำบลพุแค อำเภอเฉลิมพระเกียรติ",
        "open_time": "08:00-18:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/SXbGC3ABX2PNL7pK7"
    },
    {
        "name": "น้ำตกเจ็ดสาวน้อย",
        "category": ["ธรรมชาติ", "น้ำตก"],
        "district": "มวกเหล็ก",
        "description": "น้ำตก 7 ชั้น เล่นน้ำได้ เหมาะกับครอบครัวและนักท่องเที่ยวเชิงธรรมชาติ",
        "address": "อุทยานแห่งชาติน้ำตกเจ็ดสาวน้อย อำเภอมวกเหล็ก",
        "open_time": "08:00-17:00",
        "entry_fee": "ผู้ใหญ่ 40 บาท เด็ก 20 บาท",
        "best_time": "ก.ค.-ต.ค.",
        "location": "https://maps.app.goo.gl/Kn1okebDEugUoB4CA"
    },
    {
        "name": "วัดป่าสว่างบุญ",
        "category": ["วัด"],
        "district": "แก่งคอย",
        "description": "วัดดังเจดีย์ 500 ยอด ภายในบรรจุพระบรมสารีริกธาตุ วัดบรรยากาศสงบ",
        "address": "6/2 ตำบลชะอม อำเภอแก่งคอย",
        "open_time": "06:00-17:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/m4SfGN8hpW152feU9"
    },
    {
        "name": "น้ำตกสามหลั่น",
        "category": ["ธรรมชาติ", "น้ำตก"],
        "district": "เมืองสระบุรี",
        "description": "น้ำตก 3 ชั้นในอุทยานแห่งชาติ น้ำตกสวย มีความสำคัญทางประวัติศาสตร์สงครามโลกครั้งที่ 2",
        "address": "53 หมู่ 1 ตำบลหนองปลาไหล อำเภอเมือง",
        "open_time": "08:00-18:00",
        "entry_fee": "ผู้ใหญ่ 20 บาท เด็ก 10 บาท",
        "best_time": "ก.ค.-ต.ค.",
        "location": "https://maps.app.goo.gl/PkRAsWYAXQR1Ac1P7"
    },
    {
        "name": "ไร่องุ่นภูนวพันธุ์",
        "category": ["ฟาร์ม"],
        "district": "มวกเหล็ก",
        "description": "ไร่องุ่นชื่อดัง ผลิตองุ่นสด องุ่นแปรรูป และน้ำองุ่น ชิมได้ ซื้อได้",
        "address": "151 หมู่ 12 ตำบลหนองย่างเสือ อำเภอมวกเหล็ก",
        "open_time": "จ.-ศ. 08:00-15:30, ส.-อา. 08:00-17:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ธ.ค.-ก.พ. ช่วงองุ่นออกผล",
        "location": "https://maps.app.goo.gl/igo6KYypfd39sCB6A"
    },
    {
        "name": "ตลาดน้ำโบราณบ้านต้นตาล",
        "category": ["ตลาด"],
        "district": "เสาไห้",
        "description": "ตลาดนัดชุมชนไทยวน สินค้าพื้นเมือง อาหารเหนือ ผ้าทอ งานหัตถกรรม อาหารโบราณ",
        "address": "ตำบลบ้านต้นตาล อำเภอเสาไห้",
        "open_time": "เสาร์-อาทิตย์",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "เช้า-บ่าย",
        "location": "https://maps.app.goo.gl/YEvNhLRLYw84awH2A"
    },
    {
        "name": "ฟาร์มสายทอง",
        "category": ["ฟาร์ม"],
        "district": "มวกเหล็ก",
        "description": "ฟาร์มมัลเบอร์รี่ปลอดสาร ให้ชิมสดๆ จากต้น เรียนรู้การปลูกและแปรรูปเป็นผลิตภัณฑ์สุขภาพ",
        "address": "อำเภอมวกเหล็ก จังหวัดสระบุรี",
        "open_time": "สอบถามฟาร์ม",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "พ.ย.-ก.พ. ผลมัลเบอร์รี่สุก",
        "location": "https://maps.app.goo.gl/f1Gmq1udNubZa6Kz5"
    },
    {
        "name": "วัดถ้ำกระบอก",
        "category": ["วัด"],
        "district": "พระพุทธบาท",
        "description": "วัดชื่อดังที่เคยเป็นสถานบำบัดผู้ติดยาเสพติด มีพระหินลาวาโดดเด่น",
        "address": "ถนนพหลโยธิน ตำบลขุนโขลน อำเภอพระพุทธบาท",
        "open_time": "08:00-15:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/e1kQEB3yAEoQibbz5"
    },
    {
        "name": "เจ็ดคด-โป่งก้อนเส้า",
        "category": ["ธรรมชาติ", "แคมป์ปิ้ง"],
        "district": "แก่งคอย",
        "description": "ศูนย์ศึกษาธรรมชาติ เหมาะสำหรับกางเต็นท์ เดินป่า น้ำตก และสัมผัสป่าเขา",
        "address": "ตำบลท่ามะปราง อำเภอแก่งคอย",
        "open_time": "08:00-18:00",
        "entry_fee": "ไม่มีค่าเข้า (ค่าเต็นท์ต่างหาก)",
        "best_time": "พ.ย.-ก.พ.",
        "location": "https://maps.app.goo.gl/HQ6etr6e8fDRSByk7"
    },
    {
        "name": "น้ำตกโกรกอีดก",
        "category": ["ธรรมชาติ", "น้ำตก"],
        "district": "แก่งคอย",
        "description": "น้ำตก 8 ชั้น สูง 350 เมตร สวยที่สุดที่ชั้น 6-7 ระหว่างทางมีเห็ดแชมเปญและต้นกระเหรี่ยงยักษ์",
        "address": "ตำบลท่ามะปราง อำเภอแก่งคอย",
        "open_time": "08:00-18:00",
        "entry_fee": "ผู้ใหญ่ 40 บาท เด็ก 20 บาท",
        "best_time": "ก.ค.-ต.ค.",
        "location": "https://goo.gl/maps/jhuVrDoWfx2baEqy5"
    },
    {
        "name": "อุโมงค์รถไฟพระพุทธฉาย",
        "category": ["แลนด์มาร์ก"],
        "district": "วิหารแดง",
        "description": "อุโมงค์รถไฟยาวอันดับ 2 ของประเทศไทย จุดถ่ายรูปยอดนิยม บรรยากาศเงียบสงบ",
        "address": "ระหว่างสถานีวิหารแดง-บุใหญ่ จังหวัดสระบุรี",
        "open_time": "ไม่มีข้อมูล",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/6GzeixmQ2PxPUZ326"
    },
    {
        "name": "ฟาร์มโคนมไทย-เดนมาร์ก",
        "category": ["ฟาร์ม"],
        "district": "มวกเหล็ก",
        "description": "ฟาร์มโคนมแห่งแรกของไทย เปิดให้นักท่องเที่ยวชม ทดลองรีดนม และซื้อผลิตภัณฑ์นมสด",
        "address": "อำเภอมวกเหล็ก จังหวัดสระบุรี",
        "open_time": "08:00-16:30",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/XnQx6qPJg6P1MocX7"
    },
    {
        "name": "น้ำตกโพธิ์หินดาด",
        "category": ["ธรรมชาติ", "น้ำตก"],
        "district": "แก่งคอย",
        "description": "น้ำตกชั้นเดียวในอุทยานแห่งชาติน้ำตกสามหลั่น บรรยากาศเงียบสงบ",
        "address": "ตำบลท่ามะปราง อำเภอแก่งคอย",
        "open_time": "08:00-17:00",
        "entry_fee": "ผู้ใหญ่ 20 บาท เด็ก 10 บาท",
        "best_time": "ก.ค.-ต.ค.",
        "location": "https://maps.app.goo.gl/4giiFZkEzYXmJMBeA"
    },
    {
        "name": "วัดพระพุทธฉาย",
        "category": ["วัด"],
        "district": "เมืองสระบุรี",
        "description": "วัดเก่าแก่กว่า 400 ปี มีรอยพระพุทธฉายบนหน้าผาและรอยพระพุทธบาทที่กรมศิลปากรบูรณะ",
        "address": "ตำบลหนองปลาไหล อำเภอเมืองสระบุรี",
        "open_time": "08:00-17:00",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/YttdjReCLUfTB8dk8"
    },
    {
        "name": "อนุสรณ์สถานสงครามโลกครั้งที่ 2 เขาแดง",
        "category": ["แลนด์มาร์ก", "ประวัติศาสตร์"],
        "district": "บ้านหมอ",
        "description": "อนุสรณ์สถานที่ทหารญี่ปุ่นสร้างสมัยสงครามโลกครั้งที่ 2 มีอุโมงค์บัญชาการและสนามเพลาะ",
        "address": "ตำบลบ้านหมอ อำเภอบ้านหมอ จังหวัดสระบุรี",
        "open_time": "ไม่มีข้อมูล",
        "entry_fee": "ไม่มีค่าเข้า",
        "best_time": "ตลอดปี",
        "location": "https://maps.app.goo.gl/pR8SrcM3Vp6yy1if8"
    }
]


In [19]:
# =========================================
# SARABURIBOT DATASET GENERATOR (FULL)
# ต้องมี places_info นิยามไว้ก่อนรันเซลล์นี้
# =========================================

import re, random, itertools
import pandas as pd
from collections import defaultdict

random.seed(42)

# ---------- Helper: category aliases (ภาษาคนไทยใช้จริง) ----------
category_alias = {
    "น้ำตก": ["น้ำตก", "เล่นน้ำ", "ลำธาร", "ที่เย็น ๆ", "จุดเล่นน้ำ", "ธารน้ำตก"],
    "วัด": ["วัด", "ไหว้พระ", "ทำบุญ", "วัดสวย", "วัดดัง", "สักการะ"],
    "ฟาร์ม": ["ฟาร์ม", "ไร่", "ฟาร์มโคนม", "สวน", "ฟาร์มสัตว์"],
    "ตลาด": ["ตลาด", "ตลาดน้ำ", "แหล่งของกิน", "ของฝาก", "ถนนคนเดิน"],
    "ดอกไม้": ["ดอกไม้", "ทุ่งดอกไม้", "คอสมอส", "เบญจมาศ", "ทุ่งดอกสวย ๆ"],
    "ธรรมชาติ": ["ธรรมชาติ", "วิวเขา", "อุทยาน", "ป่าเขา", "จุดชมวิว"],
    "แคมป์ปิ้ง": ["แคมป์ปิ้ง", "กางเต็นท์", "ค้างคืน", "ตั้งแคมป์"],
    "แลนด์มาร์ก": ["แลนด์มาร์ก", "จุดถ่ายรูป", "ถ่ายรูปสวย", "ที่เช็คอิน"],
    "ประวัติศาสตร์": ["ประวัติศาสตร์", "โบราณสถาน", "สงครามโลก", "ร่องรอยอดีต"],
}

# ตัดเฉพาะ alias ที่เกี่ยวข้องกับหมวดที่มีจริง
all_categories = sorted(set(itertools.chain.from_iterable(p["category"] for p in places_info)))
cat_to_alias = {c: category_alias.get(c, [c]) for c in all_categories}

# ---------- Helper: nearby (ตามอำเภอเดียวกัน) ----------
by_district = defaultdict(list)
for p in places_info:
    by_district[p["district"]].append(p["name"])

def compute_nearby(name: str):
    place = next((x for x in places_info if x["name"] == name), None)
    if not place:
        return ""
    district = place["district"]
    others = [n for n in by_district[district] if n != name]
    return " / ".join(others) if others else f"ในอำเภอ {district} ยังไม่พบที่เที่ยวอื่นในฐานข้อมูล"

# ---------- Helper: augmentation ----------
polite_particles = ["", "ครับ", "ค่ะ", "คับ", "คะ", "นะ", "น้า", "หน่อย", "หน่อยย", "ดิ", "ด้วย", "ที"]
ask_variants = [("ไหม", "มั้ย"), ("ได้ไหม", "ได้มั้ย"), ("หรือเปล่า", "รึเปล่า")]
emoji_pool = ["😊","✨","🙏","🌄","🏞️","📍","🗺️","💦","⛺","⛩️","🛕","📸","🌸","🍇","🥛","🛍️"]

def elongate_word(text, words=("หน่อย", "สวย", "อยาก", "แนะนำ")):
    t = text
    for w in words:
        if w in t and random.random() < 0.35:
            t = t.replace(w, w + random.choice(["ย", "ยย", "มาก"]))
    return t

def thai_typo_variants(text):
    # เคสสะกดผิดที่เจอบ่อย
    variants = set([text])
    rep = {
        "แนะนำ": ["แนะนํา"],  # นิคหิตบนสระอำ
        "ที่เที่ยว": ["ทีเที่ยว", "ที่เทียว"],
        "เปิดกี่โมง": ["เปิดกี่โมงง", "เปิดกี่โมงคะ"],
        "อยู่ที่ไหน": ["อยู่ไหน", "อยู่ที่ใหน"],
        "ค่าเข้า": ["ค่้าเข้า", "ค่าเข้"],
        "ใกล้": ["ไกล้"],  # ผิดบ่อย
        "เจ็ดสาวน้อย": ["7 สาวน้อย", "เจ็ด สาวน้อย", "7สาวน้อย"],
    }
    for k, alts in rep.items():
        if k in text:
            for a in alts:
                variants.add(text.replace(k, a))
    # แปลง "ไหม" <-> "มั้ย"
    for a, b in ask_variants:
        if a in text: variants.add(text.replace(a, b))
        if b in text: variants.add(text.replace(b, a))
    return list(variants)

def add_particles_and_emojis(text):
    out = set()
    for p1 in polite_particles:
        t1 = (text + (" " + p1 if p1 else "")).strip()
        t1 = elongate_word(t1)
        out.add(t1)
        # เติมอีโมจิบ้าง ๆ
        if random.random() < 0.35:
            out.add(t1 + " " + random.choice(emoji_pool))
    return list(out)

def augment_question(base_q, max_variants=6):
    pool = set()
    for v in thai_typo_variants(base_q):
        for v2 in add_particles_and_emojis(v):
            pool.add(v2)
    # จำกัดจำนวนเพื่อไม่ให้ชุดข้อมูลโตเกินไป
    pool = list(pool)
    random.shuffle(pool)
    return [base_q] + pool[:max_variants]

# ---------- เริ่มสร้าง QA ----------
qa_pairs = []

# 1) พื้นฐาน/ทักทาย
basic_intents = {
    "greeting": [
        "สวัสดี", "หวัดดี", "hello", "hi", "ไง", "เฮ้", "มีใครอยู่ไหม", "ดีครับ", "ดีค่า", "ดีจ้า"
    ],
    "goodbye": [
        "บาย", "ลาก่อน", "see you", "ไปก่อนนะ", "เจอกันใหม่", "สวัสดีครับ บาย", "ไปละ"
    ],
    "ask_name": [
        "คุณชื่ออะไร", "บอทชื่ออะไร", "who are you", "ชื่ออะไรครับ", "ชื่อของคุณคืออะไร"
    ],
    "capability": [
        "บอทนี้ทำอะไรได้บ้าง", "คุณช่วยอะไรได้", "คุณทำอะไรได้", "คุณตอบเรื่องอะไรได้", "ช่วยอะไรได้บ้าง",
        "ทำอะไรให้ได้บ้าง", "ทำไรได้บ้าง"
    ],
}
basic_responses = {
    "greeting": "สวัสดีครับ! ผมคือ SaraburiBot พร้อมแนะนำที่เที่ยวครับ",
    "goodbye": "บายครับ ไว้เจอกันใหม่ 🙏",
    "ask_name": "ผมชื่อ SaraburiBot ครับ",
    "capability": "ผมช่วยตอบเรื่องที่เที่ยวในสระบุรี เช่น วัด น้ำตก ฟาร์ม ตลาด ครับ",
}
for intent, qs in basic_intents.items():
    for q in qs:
        for qv in augment_question(q, max_variants=3):
            qa_pairs.append((qv, intent, basic_responses[intent]))

# 2) หมวดหมู่ (รองรับ alias และการถามหลายแบบ)
for cat, aliases in cat_to_alias.items():
    # รายการสถานที่ในหมวดนี้
    names_in_cat = [p["name"] for p in places_info if cat in p["category"]]
    if not names_in_cat:
        continue
    answer = " / ".join(names_in_cat)
    base_templates = [
        "อยากเที่ยว{a}", "มี{a}แนะนำไหม", "แนะนำ{a}หน่อย", "{a}อื่นล่ะ", "มีที่เที่ยว{a}อะไรบ้าง",
        "อยากได้ที่เที่ยวแนว{a}", "ขอ{a}สวย ๆ หน่อย", "แถวนี้มี{a}ไหม", "{a}ยอดนิยมมีที่ไหนบ้าง",
        "อยากได้{a}ที่คนชอบไป", "{a}ที่ไม่ควรพลาด", "{a}", "มี{a}อีกไหม", "ขอพิกัด{a}", "{a}ในสระบุรี",
        "{a}ใกล้กรุงเทพ", "อยากได้ลิสต์{a}", "รวม{a}ดัง ๆ หน่อย", "มี{a}ไปกับเด็กได้ไหม", "อยากได้{a}ถ่ายรูปสวย"
    ]
    for a in aliases:
        for tmpl in base_templates:
            q = tmpl.format(a=a)
            for qv in augment_question(q, max_variants=3):
                qa_pairs.append((qv, f"recommend_{cat}", answer))

# 3) รายสถานที่
for p in places_info:
    name = p["name"]
    district = p["district"]
    desc = p["description"]
    open_time = p["open_time"]
    entry_fee = p["entry_fee"]
    best_time = p["best_time"]
    address = p["address"]
    location = p["location"]

    # 3.1 detail
    detail_templates = [
        "{n} คืออะไร", "{n} คือที่แบบไหน", "ช่วยเล่าเกี่ยวกับ {n}", "{n} มีอะไรน่าสนใจบ้าง",
        "รายละเอียด {n}", "{n}", "ไป {n} ดีไหม", "รีวิว {n} หน่อย", "{n} เป็นยังไงบ้าง"
    ]
    for t in detail_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"detail_{name}", f"{desc}"))

    # 3.2 district / address
    district_templates = [
        "{n} อยู่ที่ไหน", "{n} อยู่เขตไหน", "{n} อำเภออะไร", "{n} อยู่จังหวัดไหน", "แถวไหน {n}"
    ]
    for t in district_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"ask_district_{name}", f"อยู่ที่อำเภอ {district} จังหวัดสระบุรี 📍 ที่อยู่: {address}"))

    # 3.3 open time (รวม “ปิดกี่โมง/วันนี้เปิดไหม”)
    open_templates = [
        "{n} เปิดกี่โมง", "{n} เวลาเปิดปิด", "{n} เปิดวันไหนบ้าง", "{n} เวลาเปิดทำการ",
        "{n} ปิดกี่โมง", "วันนี้ {n} เปิดไหม", "{n} เปิดทุกวันไหม"
    ]
    for t in open_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"ask_open_time_{name}", f"เวลาเปิด-ปิด: {open_time} (แนะนำตรวจสอบเพจ/โทรก่อนเดินทาง)"))

    # 3.4 entry fee (รวม “เสียเงินไหม/ค่าบัตร/ราคา”)
    fee_templates = [
        "{n} ค่าเข้าเท่าไร", "{n} เสียเงินไหม", "ไป {n} ต้องจ่ายเท่าไร", "ค่าเข้าชม {n}",
        "บัตรเข้า {n} ราคาเท่าไร", "{n} ฟรีไหม"
    ]
    for t in fee_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"ask_entry_fee_{name}", f"{entry_fee}"))

    # 3.5 best time
    best_templates = [
        "{n} ควรไปเมื่อไร", "{n} ฤดูไหนเหมาะ", "ไป {n} เดือนไหนสวย", "{n} ไปช่วงไหนดีที่สุด"
    ]
    for t in best_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"ask_best_time_{name}", f"{best_time}"))

    # 3.6 map / พิกัด
    map_templates = [
        "พิกัด {n}", "แผนที่ {n}", "map {n}", "โลเคชัน {n}", "ขอ google maps {n}", "ทางไป {n}"
    ]
    for t in map_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"ask_location_{name}", f"พิกัด: {location} 📍 ที่อยู่: {address}"))

    # 3.7 nearby
    nearby_resp = compute_nearby(name)
    nearby_templates = [
        "ที่เที่ยวใกล้ {n} มีอะไรบ้าง", "ใกล้ {n} ไปไหนต่อดี", "แถว {n} มีอะไร",
        "อยากได้ที่เที่ยวต่อจาก {n}", "จาก {n} ไปไหนดี", "ใกล้ {n} แนะนำหน่อย"
    ]
    for t in nearby_templates:
        for qv in augment_question(t.format(n=name), max_variants=3):
            qa_pairs.append((qv, f"ask_nearby_{name}", f"{nearby_resp}"))

# ---------------- Save & Summaries ----------------
df = pd.DataFrame(qa_pairs, columns=["question", "intent", "response"]).drop_duplicates().reset_index(drop=True)
df.to_excel("qa_dataset.xlsx", index=False)
df.to_csv("qa_dataset.csv", index=False, encoding="utf-8-sig")

print("✅ Dataset created!")
print("Rows:", len(df))
print("Unique intents:", df['intent'].nunique())

# แสดงสรุป 10 intents ตัวอย่าง
print("\n🔎 Intent distribution (top 10):")
print(df['intent'].value_counts().head(10))

# ดูตัวอย่างบางแถว
display(df.sample(15, random_state=42))


✅ Dataset created!
Rows: 6853
Unique intents: 153

🔎 Intent distribution (top 10):
intent
recommend_น้ำตก            470
recommend_วัด              469
recommend_ตลาด             392
recommend_ดอกไม้           389
recommend_ธรรมชาติ         386
recommend_ฟาร์ม            386
recommend_แลนด์มาร์ก       317
recommend_ประวัติศาสตร์    311
recommend_แคมป์ปิ้ง        310
greeting                    40
Name: count, dtype: int64


,question,intent,response
469,มีทุ่งดอกสวย ๆอีกมั้ย นะ 😊,recommend_ดอกไม้,สวนบิ๊กเต้ / ทุ่งสิริสมัย
3855,แถว วัดพระพุทธบาท มีอะไร ค่ะ,ask_nearby_วัดพระพุทธบาท,วัดถ้ำกระบอก
4776,วัดป่าสว่างบุญ ปิดกี่โมง ที,ask_open_time_วัดป่าสว่างบุญ,เวลาเปิด-ปิด: 06:00-17:00 (แนะนำตรวจสอบเพจ/โทร...
3027,อยากได้กางเต็นท์ที่คนชอบไป,recommend_แคมป์ปิ้ง,เจ็ดคด-โป่งก้อนเส้า
4635,น้ำตกเจ็ดสาวน้อย ฟรีไหม,ask_entry_fee_น้ำตกเจ็ดสาวน้อย,ผู้ใหญ่ 40 บาท เด็ก 20 บาท
5337,ขอ google maps ตลาดน้ำโบราณบ้านต้นตาล คับ 🌄,ask_location_ตลาดน้ำโบราณบ้านต้นตาล,พิกัด: https://maps.app.goo.gl/YEvNhLRLYw84awH...
2900,อยากได้ลิสต์สักการะ ด้วย 🙏,recommend_วัด,วัดพระพุทธบาท / วัดป่าสว่างบุญ / วัดถ้ำกระบอก ...
4523,อยากได้ที่เทียวต่อจาก สวนพฤกษศาสตร์พุแค น้า,ask_nearby_สวนพฤกษศาสตร์พุแค,ตลาดหัวปลี
1393,อยากได้เล่นน้ำที่คนชอบไป น้า,recommend_น้ำตก,น้ำตกเจ็ดสาวน้อย / น้ำตกสามหลั่น / น้ำตกโกรกอี...
3235,แลนด์มาร์กอื่นล่ะ,recommend_แลนด์มาร์ก,อุโมงค์รถไฟพระพุทธฉาย / อนุสรณ์สถานสงครามโลกคร...


In [20]:
# ================================
# CELL 5 - Better Thai Intent Model
# ================================
import re, numpy as np, pandas as pd, joblib, warnings
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

warnings.filterwarnings("ignore")

# ------- 1) Thai text normalizer -------
def normalize_thai(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.strip()

    # ลดการยืดตัวอักษรเกินเหตุ เช่น หน่อยยย → หน่อย
    t = re.sub(r"(.)\1{2,}", r"\1\1", t)

    # เว้นวรรคซ้ำซ้อน
    t = re.sub(r"\s+", " ", t)

    # ปรับคำ/การสะกดที่เจอบ่อย
    subs = {
        "มั้ย": "ไหม",
        "ได้มั้ย": "ได้ไหม",
        "รึเปล่า": "หรือเปล่า",
        "แนะนํา": "แนะนำ",
        "ไกล้": "ใกล้",
        "ที่เทียว": "ที่เที่ยว",
        "ใหน": "ไหน",
        "ครับๆ": "ครับ",
        "ค่ะๆ": "ค่ะ",
    }
    for a, b in subs.items():
        t = t.replace(a, b)

    # ตัดอีโมจิ/สัญลักษณ์ที่ไม่ช่วยมากเกินไป (เก็บได้ถ้าอยาก)
    t = re.sub(r"[^\w\s\u0E00-\u0E7F/:-]", " ", t)  # เก็บไทย/อังกฤษ/ตัวเลข/เครื่องหมายพื้นฐาน
    t = re.sub(r"\s+", " ", t).strip()
    return t

class ThaiNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return [normalize_thai(x) for x in X]

# ------- 2) Prepare data -------
X_text = df["question"].astype(str)
y = df["intent"].astype(str)

# พยายาม stratify ถ้าเป็นไปได้ (ถ้าคลาสเยอะ/ข้อมูลน้อยจะ fallback อัตโนมัติ)
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X_text, y, test_size=0.25, random_state=42, stratify=y
    )
except ValueError:
    X_train, X_test, y_train, y_test = train_test_split(
        X_text, y, test_size=0.30, random_state=42
    )

# ------- 3) Featurization: char + word n-grams -------
char_v = TfidfVectorizer(
    analyzer="char", ngram_range=(3, 6), min_df=2, sublinear_tf=True
)
word_v = TfidfVectorizer(
    analyzer="word", ngram_range=(1, 2), min_df=1, sublinear_tf=True,
    token_pattern=r"(?u)\b\w+\b"  # เวิร์คกับไทยแบบไม่ตัดคำ (ใช้สเปซ/ตัวอักษร)
)

model = Pipeline([
    ("norm", ThaiNormalizer()),
    ("feats", FeatureUnion([
        ("char", char_v),
        ("word", word_v),
    ])),
    ("clf", LinearSVC(C=2.0, class_weight="balanced", max_iter=5000))
])

# ------- 4) Train -------
model.fit(X_train, y_train)

# ------- 5) Evaluate -------
pred_tr = model.predict(X_train)
pred_te = model.predict(X_test)

train_acc = accuracy_score(y_train, pred_tr)
test_acc  = accuracy_score(y_test,  pred_te)
macro_f1  = f1_score(y_test, pred_te, average="macro")

print(f"✅ Train Accuracy : {train_acc:.4f}")
print(f"✅ Test  Accuracy : {test_acc:.4f}")
print(f"✅ Test  Macro-F1 : {macro_f1:.4f}\n")

# รายงานรวม (อาจยาว)
print("📊 Classification report (ย่อบางส่วน):")
report = classification_report(y_test, pred_te, output_dict=True, zero_division=0)
# แสดงสรุปเฉพาะ macro avg + weighted avg
for k in ["macro avg", "weighted avg"]:
    row = report.get(k, {})
    print(f"  - {k:12s}  precision={row.get('precision',0):.3f}  recall={row.get('recall',0):.3f}  f1={row.get('f1-score',0):.3f}")
print()

# หา intents ที่งงบ่อย (F1 ต่ำสุด)
per_intent = []
for lbl, stats in report.items():
    if lbl in ["accuracy", "macro avg", "weighted avg"]:
        continue
    per_intent.append((lbl, stats.get("f1-score", 0.0), int(stats.get("support", 0))))
per_intent.sort(key=lambda x: (x[1], x[2]))  # f1 จากน้อยไปมาก

print("🔻 10 intents ที่บอทงงบ่อย (ตาม F1-score):")
for lbl, f1v, sup in per_intent[:10]:
    print(f"  - {lbl:40s}  f1={f1v:.3f}  support={sup}")

# ------- 6) Save model -------
joblib.dump(model, "chatbot_model.pkl")
print("\n💾 Saved model -> chatbot_model.pkl")

# (ออปชัน) เซฟ intent->responses เผื่อใช้ในโหมด console
intent_to_responses = df.groupby("intent")["response"].apply(list).to_dict()
joblib.dump(intent_to_responses, "responses_dict.pkl")
print("💾 Saved responses -> responses_dict.pkl")


✅ Train Accuracy : 1.0000
✅ Test  Accuracy : 0.9988
✅ Test  Macro-F1 : 0.9978

📊 Classification report (ย่อบางส่วน):
  - macro avg     precision=0.998  recall=0.998  f1=0.998
  - weighted avg  precision=0.999  recall=0.999  f1=0.999

🔻 10 intents ที่บอทงงบ่อย (ตาม F1-score):
  - ask_location_ทุ่งสิริสมัย                 f1=0.800  support=6
  - ask_nearby_ทุ่งสิริสมัย                   f1=0.923  support=6
  - detail_ทุ่งสิริสมัย                       f1=0.947  support=9
  - ask_best_time_ตลาดน้ำโบราณบ้านต้นตาล      f1=1.000  support=3
  - ask_best_time_วัดป่าสว่างบุญ              f1=1.000  support=3
  - ask_best_time_สวนพฤกษศาสตร์พุแค           f1=1.000  support=3
  - ask_best_time_อนุสรณ์สถานสงครามโลกครั้งที่ 2 เขาแดง  f1=1.000  support=3
  - ask_best_time_ตลาดหัวปลี                  f1=1.000  support=4
  - ask_best_time_ทุ่งสิริสมัย                f1=1.000  support=4
  - ask_best_time_น้ำตกสามหลั่น               f1=1.000  support=4

💾 Saved model -> chatbot_model.pkl
💾 Saved responses

In [21]:
# =========================
# CELL 6 — Better Chat Loop
# =========================
import joblib, random, numpy as np
from collections import defaultdict

# 1) โหลดโมเดล + responses dict (ไม่พึ่ง df)
try:
    model = joblib.load("chatbot_model.pkl")
    responses_dict = joblib.load("responses_dict.pkl")
    print("✅ Loaded model & responses_dict")
except Exception as e:
    print("❌ ไม่พบไฟล์โมเดล/รีสปอนส์ กรุณารัน Cell 5 ก่อน:", e)
    model, responses_dict = None, {}

# 2) เตรียมดัชนีจาก places_info (หมวด → รายชื่อสถานที่, อำเภอ → รายชื่อสถานที่)
cat_to_places = defaultdict(list)
name_to_place = {}
district_to_places = defaultdict(list)

for p in places_info:
    name_to_place[p["name"]] = p
    district_to_places[p["district"]].append(p["name"])
    for c in p.get("category", []):
        if p["name"] not in cat_to_places[c]:
            cat_to_places[c].append(p["name"])

def get_nearby(place_name: str) -> str:
    "แนะนำที่เที่ยวในอำเภอเดียวกันกับ place_name"
    pl = name_to_place.get(place_name)
    if not pl:
        return f"ไม่พบ {place_name} ในฐานข้อมูล"
    d = pl["district"]
    others = [n for n in district_to_places[d] if n != place_name]
    return (
        f"ที่เที่ยวใกล้ {place_name} (อ.{d})\n- " + "\n- ".join(others)
        if others else
        f"ที่เที่ยวใกล้ {place_name} (อ.{d}) ยังไม่มีข้อมูลอื่นบันทึกไว้ครับ"
    )

def format_bullets(names, k=4) -> str:
    """สุ่มแนะนำ 2–4 ที่แบบ bullet list"""
    if not names:
        return "ยังไม่มีข้อมูลในหมวดนี้"
    k = min(max(2, min(k, len(names))), len(names))
    choice = random.sample(names, k) if len(names) > k else names
    return "- " + "\n- ".join(choice)

def predict_intent_with_margin(text: str):
    """ทาย intent พร้อม margin (ต่างของคะแนนอันดับ 1 และ 2) เพื่อกันเคสไม่มั่นใจ"""
    intent = model.predict([text])[0]
    margin = None
    try:
        scores = model.decision_function([text])  # LinearSVC ให้คะแนนต่อคลาส
        if scores.ndim == 2 and scores.shape[1] >= 2:
            s = scores[0]
            top2 = np.sort(s)[-2:]     # คะแนนสูงสุดสองอันดับ
            margin = float(top2[1] - top2[0])
    except Exception:
        pass
    return intent, margin

def chatbot_response(user_input: str) -> str:
    if not model or not responses_dict:
        return "⚠️ ยังไม่มีโมเดลพร้อมใช้งาน กรุณารัน Cell 5 ก่อนนะครับ"

    text = (user_input or "").strip()
    if not text:
        return "พิมพ์คำถามมาได้เลยครับ 😄"

    # 3) ทาย intent + ตรวจความมั่นใจ (margin guard)
    intent, margin = predict_intent_with_margin(text)

    # ถ้าความมั่นใจต่ำมาก ให้ชวนระบุหมวด/ชื่อสถานที่เพิ่มเติม
    if margin is not None and margin < 0.12:
        return "ผมไม่ค่อยมั่นใจคำถามนี้ 🤔 ลองระบุหมวด (เช่น น้ำตก/วัด/ฟาร์ม) หรือชื่อสถานที่เพิ่มหน่อยนะครับ 😊"

    # 4) แฮนด์เมด rules: recommend_หมวด → ตอบหลายที่
    if intent.startswith("recommend_"):
        cat = intent.replace("recommend_", "")
        places = cat_to_places.get(cat, [])
        if places:
            return f"แนะนำที่เที่ยวแนว **{cat}**:\n{format_bullets(places, k=4)}"
        # ถ้าไม่พบในดัชนี ลองใช้ responses_dict เดิม (เผื่อมีสตริง list)
        res = responses_dict.get(intent, [])
        if res:
            return random.choice(res)

    # ask_nearby_ชื่อสถานที่ → ใช้ดัชนีอำเภอ
    if intent.startswith("ask_nearby_"):
        place = intent.replace("ask_nearby_", "")
        return get_nearby(place)

    # 5) ทั่วไป: ดึงจาก responses_dict
    reslist = responses_dict.get(intent)
    if reslist:
        return random.choice(reslist)

    # 6) Fallback
    return "ขอโทษครับ ผมยังไม่เข้าใจคำถามนี้ 😅 ลองพิมพ์ชื่อสถานที่หรือหมวด (น้ำตก/วัด/ฟาร์ม) ดูนะครับ"

# ------------------ Quick tests ------------------
tests = [
    "สวัสดี",
    "มีน้ำตกแนะนำไหม",
    "อยากเที่ยวฟาร์ม",
    "วัดพระพุทธบาท อยู่ที่ไหน",
    "ใกล้ น้ำตกเจ็ดสาวน้อย ไปไหนต่อดี",
    "พิกัด วัดพระพุทธฉาย",
    "ตลาด อะไรบ้าง",
    "แนะนำดอกไม้หน่อยยย 🌸",
]
for q in tests:
    print("You:", q)
    print("Bot:", chatbot_response(q))
    print("-" * 60)


✅ Loaded model & responses_dict
You: สวัสดี
Bot: สวัสดีครับ! ผมคือ SaraburiBot พร้อมแนะนำที่เที่ยวครับ
------------------------------------------------------------
You: มีน้ำตกแนะนำไหม
Bot: แนะนำที่เที่ยวแนว **น้ำตก**:
- น้ำตกเจ็ดสาวน้อย
- น้ำตกสามหลั่น
- น้ำตกโกรกอีดก
- น้ำตกโพธิ์หินดาด
------------------------------------------------------------
You: อยากเที่ยวฟาร์ม
Bot: แนะนำที่เที่ยวแนว **ฟาร์ม**:
- สวนบิ๊กเต้
- ไร่องุ่นภูนวพันธุ์
- ฟาร์มสายทอง
- ฟาร์มโคนมไทย-เดนมาร์ก
------------------------------------------------------------
You: วัดพระพุทธบาท อยู่ที่ไหน
Bot: อยู่ที่อำเภอ พระพุทธบาท จังหวัดสระบุรี 📍 ที่อยู่: ตำบลขุนโขลน อำเภอพระพุทธบาท จังหวัดสระบุรี
------------------------------------------------------------
You: ใกล้ น้ำตกเจ็ดสาวน้อย ไปไหนต่อดี
Bot: ที่เที่ยวใกล้ น้ำตกเจ็ดสาวน้อย (อ.มวกเหล็ก)
- สวนบิ๊กเต้
- ไร่องุ่นภูนวพันธุ์
- ฟาร์มสายทอง
- ฟาร์มโคนมไทย-เดนมาร์ก
------------------------------------------------------------
You: พิกัด วัดพระพุทธฉาย
Bot: พิกัด: https://maps.app

In [22]:
%%writefile app.py
import os
import re
import random
import numpy as np
import pandas as pd
import streamlit as st
import joblib

# =========================
# ThaiNormalizer class (ต้องมาก่อน joblib.load) — ให้ตรงกับตอนเทรน
# =========================
from sklearn.base import BaseEstimator, TransformerMixin

def _normalize_thai_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.strip()
    t = re.sub(r"(.)\1{2,}", r"\1\1", t)   # ลดการยืดตัวอักษร
    t = re.sub(r"\s+", " ", t)             # ช่องว่างเดียว
    subs = {
        "มั้ย": "ไหม", "ได้มั้ย": "ได้ไหม", "รึเปล่า": "หรือเปล่า",
        "แนะนํา": "แนะนำ", "ไกล้": "ใกล้", "ที่เทียว": "ที่เที่ยว", "ใหน": "ไหน",
    }
    for a, b in subs.items():
        t = t.replace(a, b)
    return t

class ThaiNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):   return [_normalize_thai_text(x) for x in X]

# --- safe rerun for all Streamlit versions ---
def _rerun():
    if hasattr(st, "rerun"):
        st.rerun()
    elif hasattr(st, "experimental_rerun"):
        st.experimental_rerun()

# =========================
# App Config
# =========================
os.environ["STREAMLIT_BROWSER_GATHERUSAGESTATS"] = "false"
st.set_page_config(page_title="SaraburiBot", page_icon="🌄", layout="centered")

# ===== CSS: Dark + Messenger-like + chip grid (equal height) =====
st.markdown("""
<style>
/* hide chrome */
#MainMenu, header, footer {visibility: hidden;}

:root{
  --bg: #0b0f14;        /* page bg */
  --panel: #0f1720;     /* panel */
  --muted: #94a3b8;     /* secondary text */
  --text: #e5e7eb;      /* body text */
  --bot: #1f2937;       /* bot bubble */
  --blue: #3b82f6;      /* user bubble */
  --chip-bg: #111827;   /* chip background */
  --chip-border: #334155;/* chip border */
  --chip-text: #e5e7eb; /* chip text */
  --chip-hover: #0b1220;/* chip hover */
  --accent: #0ea5e9;    /* accent */
}

html, body, .stApp { background: var(--bg); color: var(--text); }
[data-testid="stMarkdown"] { color: var(--text); }
a { color: var(--accent); }

/* main container */
.chat-wrap{
  max-width: 920px;
  margin: 16px auto 100px auto;
  padding: 0 12px;
}

/* top */
.title{font-size: 26px; font-weight: 700; letter-spacing:.2px; color: var(--text);}
.sub{color: var(--muted); font-size: 13px; margin-top: 2px}

/* clear button (scoped) */
.clearbtn .stButton > button{
  background: #0f172a; color:#e2e8f0; border:1px solid #1e293b;
  padding: 8px 14px; border-radius: 12px; font-weight:600;
}
.clearbtn .stButton > button:hover{
  background:#0b1220; border-color:#334155;
}

/* ========== CATEGORY GRID ========== */
/* container spacing */
.catwrap{ margin: 10px 0 14px 0; }
/* one row is a streamlit columns row; style all buttons as chips */
.catrow .stButton > button{
  width: 100%;
  height: 42px;                 /* << equal height */
  border-radius: 999px;
  border: 1px solid var(--chip-border);
  background: var(--chip-bg);
  color: var(--chip-text);
  font-size: 14px;
  padding: 6px 12px;
  display: inline-flex; align-items:center; justify-content:center;  /* << centering */
  transition: background .15s ease, border-color .15s ease, transform .02s ease;
  box-shadow: inset 0 0 0 1px rgba(255,255,255,0.02);
}
.catrow .stButton > button:hover{
  background: var(--chip-hover);
  border-color: #475569;
}
.catrow .stButton > button:active{
  transform: scale(0.995);
}

/* chat bubbles */
.msg-row { display:flex; margin: 6px 0; }
.msg { max-width: 76%; font-size: 16px; line-height: 1.5; padding: 10px 14px; border-radius: 18px; }
.left  { justify-content: flex-start; }
.right { justify-content: flex-end; }
.botbubble{
  background: var(--bot); color: var(--text);
  border-top-left-radius: 6px;
  border: 1px solid #111827;
  box-shadow: 0 2px 12px rgba(0,0,0,.25), inset 0 0 0 1px rgba(255,255,255,.02);
}
.userbubble{
  background: var(--blue); color: #fff;
  border-top-right-radius: 6px;
  border: 1px solid rgba(255,255,255,.08);
  box-shadow: 0 2px 12px rgba(0,0,0,.25), inset 0 0 0 1px rgba(255,255,255,.04);
}

/* label */
.label{ font-size: 12px; color: var(--muted); margin-bottom: 4px; display:block; }

/* bottom input area */
.inputbar{ position: fixed; left:0; right:0; bottom:12px; z-index:1000; display:flex; justify-content:center; }
.input-inner{
  width: min(920px, 94%); background: var(--panel); border:1px solid #1f2937; border-radius:16px;
  padding:10px 12px; box-shadow: 0 8px 30px rgba(0,0,0,.30), inset 0 0 0 1px rgba(255,255,255,.02);
}
</style>
""", unsafe_allow_html=True)

# =========================
# Load model & dataset
# =========================
MODEL_PATH = "chatbot_model.pkl"
DATASET_PATH = "qa_dataset.xlsx"

try:
    model = joblib.load(MODEL_PATH)
    df = pd.read_excel(DATASET_PATH)
    intent_to_response = df.groupby("intent")["response"].apply(list).to_dict()
    all_intents = list(intent_to_response.keys())
except Exception:
    st.error("โหลดโมเดล/ชุดข้อมูลไม่สำเร็จ — ตรวจไฟล์ `chatbot_model.pkl` และ `qa_dataset.xlsx` อีกครั้งครับ")
    st.stop()

# =========================
# Build indices from intents
# =========================
place_names = set()
for it in all_intents:
    for prefix in ["detail_", "ask_district_", "ask_open_time_", "ask_entry_fee_",
                   "ask_best_time_", "ask_location_", "ask_nearby_"]:
        if it.startswith(prefix):
            place_names.add(it.replace(prefix, ""))
categories = sorted([it.replace("recommend_", "") for it in all_intents if it.startswith("recommend_")])

# =========================
# Utils
# =========================
def normalize_thai(text: str) -> str:
    return _normalize_thai_text(text)

def decision_margin(text: str):
    try:
        scores = model.decision_function([text])
        if np.ndim(scores) == 2 and scores.shape[1] >= 2:
            s = scores[0]; top2 = np.sort(s)[-2:]; return float(top2[1] - top2[0])
    except Exception:
        pass
    return None

def split_candidates_from_response(resp_list):
    names = []
    for r in resp_list:
        parts = re.split(r"\s*/\s*|\s*,\s*|\s*\|\s*|•", str(r))
        for p in parts:
            p = p.strip()
            if p: names.append(p)
    return list(dict.fromkeys(names))

def bullet(names, k=4):
    if not names: return "ยังไม่มีข้อมูลในหมวดนี้"
    if len(names) > k: names = random.sample(names, k)
    return "- " + "\n- ".join(names)

def _pick_first(intent_key: str):
    lst = intent_to_response.get(intent_key, [])
    return lst[0] if lst else None

def build_place_card(name: str) -> str:
    desc     = _pick_first(f"detail_{name}")
    district = _pick_first(f"ask_district_{name}")
    open_tm  = _pick_first(f"ask_open_time_{name}")
    fee      = _pick_first(f"ask_entry_fee_{name}")
    best     = _pick_first(f"ask_best_time_{name}")
    loc      = _pick_first(f"ask_location_{name}")

    card = [f"### 📍 {name}"]
    if desc:     card.append(desc)
    if district: card.append(f"**ที่ตั้ง:** {district}")
    if open_tm:  card.append(f"**เวลาเปิด-ปิด:** {open_tm}")
    if fee:      card.append(f"**ค่าเข้า:** {fee}")
    if best:     card.append(f"**ช่วงเวลาที่เหมาะ:** {best}")
    if loc:      card.append(f"**พิกัด/แผนที่:** {loc}")
    return "\n\n".join(card)

def find_place_in_text(text: str):
    for n in sorted(place_names, key=len, reverse=True):
        if n in text: return n
    return None

def nearby_for_place(name: str):
    it = f"ask_nearby_{name}"
    res = intent_to_response.get(it, [])
    if res: return res[0]
    return "ยังไม่มีข้อมูลที่เที่ยวใกล้เคียงครับ"

# =========================
# Core answer logic
# =========================
def answer(user_text: str) -> str:
    text = normalize_thai(user_text)

    # 1) ชื่อสถานที่อย่างเดียว → การ์ดข้อมูล
    pl = find_place_in_text(text)
    if pl and len(text) <= len(pl) + 8:
        return build_place_card(pl)

    # 2) ขอที่เที่ยวใกล้...
    if pl and any(k in text for k in ["ใกล้", "แถว", "ละแวก", "ต่อจาก", "ไปไหนต่อ", "ใกล้ๆ"]):
        return nearby_for_place(pl)

    # 3) ทำนาย intent
    intent = model.predict([text])[0]
    margin = decision_margin(text)

    # 4) ถ้าไม่มั่นใจ → ตอบตรง ๆ
    if margin is not None and margin < 0.12:
        return "ผมยังไม่มั่นใจคำถามนี้ 🤔 ลองระบุชื่อสถานที่หรือหมวด (น้ำตก/วัด/ฟาร์ม) เพิ่มหน่อยนะครับ"

    # 5) หมวดหมู่ → แสดงลิสต์สั้น ๆ
    if intent.startswith("recommend_"):
        cat = intent.replace("recommend_", "")
        res_list = intent_to_response.get(intent, [])
        names = split_candidates_from_response(res_list)
        if names:
            return f"แนะนำที่เที่ยวแนว **{cat}**:\n{bullet(names, k=random.randint(3,5))}"
        if res_list:
            return random.choice(res_list)
        return "ยังไม่มีข้อมูลในหมวดนี้ครับ"

    # 6) ใกล้สถานที่ (จาก intent)
    if intent.startswith("ask_nearby_"):
        place = intent.replace("ask_nearby_", "")
        return nearby_for_place(place)

    # 7) รายละเอียดสถานที่ → การ์ด
    if intent.startswith("detail_"):
        place = intent.replace("detail_", "")
        if place in place_names:
            return build_place_card(place)

    # 8) ทั่วไป
    res = intent_to_response.get(intent)
    if res:
        return random.choice(res)

    # 9) fallback
    return "ขอโทษครับ ผมไม่ทราบคำตอบนี้ หรือข้อมูลยังไม่เพียงพอในฐานข้อมูล 🙏"

# =========================
# State & UI
# =========================
if "history" not in st.session_state:
    st.session_state.history = []

st.markdown('<div class="chat-wrap">', unsafe_allow_html=True)

# Top bar
c1, c2 = st.columns([6,2], vertical_alignment="center")
with c1:
    st.markdown('<div class="title">Messenger • SaraburiBot</div>', unsafe_allow_html=True)
    st.markdown('<div class="sub">ถามอะไรเกี่ยวกับที่เที่ยวสระบุรีได้เลย</div>', unsafe_allow_html=True)
with c2:
    st.markdown('<div class="clearbtn">', unsafe_allow_html=True)
    if st.button("🧹 ล้างประวัติ", key="clear", help="เริ่มบทสนทนาใหม่", use_container_width=True):
        st.session_state.history = []
        _rerun()
    st.markdown('</div>', unsafe_allow_html=True)

# ---------- Category chip grid (equal height) ----------
def render_category_grid(cats, cols_per_row=5):
    if not cats: return
    st.markdown('<div class="catwrap">', unsafe_allow_html=True)

    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    for row in chunks(cats, cols_per_row):
        cols = st.columns(len(row), vertical_alignment="center")
        st.markdown('<div class="catrow">', unsafe_allow_html=True)
        for i, cat in enumerate(row):
            with cols[i]:
                if st.button(f"#{cat}", key=f"cat_{cat}", use_container_width=True):
                    q = f"อยากเที่ยว{cat}"
                    a = answer(q)
                    st.session_state.history.append({"user": q, "bot": a})
                    _rerun()
        st.markdown('</div>', unsafe_allow_html=True)

    st.markdown('</div>', unsafe_allow_html=True)

render_category_grid(categories, cols_per_row=5)

# Render chat bubbles
def bubble(role: str, text: str):
    if role == "user":
        st.markdown(
            f'<div class="msg-row right"><div class="msg userbubble">'
            f'<span class="label">คุณ</span>{text}</div></div>', unsafe_allow_html=True
        )
    else:
        st.markdown(
            f'<div class="msg-row left"><div class="msg botbubble">'
            f'<span class="label">SaraburiBot</span>{text}</div></div>', unsafe_allow_html=True
        )

for chat in st.session_state.history:
    bubble("user", chat["user"])
    bubble("bot",  chat["bot"])

# Bottom input
def handle_user_text(inp: str):
    if not inp: return
    bot_text = answer(inp.strip())
    st.session_state.history.append({"user": inp, "bot": bot_text})
    _rerun()

use_chat_input = True
try:
    user_text = st.chat_input("พิมพ์ข้อความที่นี่…")
    if user_text:
        handle_user_text(user_text)
except Exception:
    use_chat_input = False

if not use_chat_input:
    st.markdown('<div class="inputbar"><div class="input-inner">', unsafe_allow_html=True)
    with st.form("chat_form", clear_on_submit=True):
        user_text = st.text_input("", "", placeholder="พิมพ์ข้อความที่นี่…")
        sent = st.form_submit_button("ส่ง")
    st.markdown('</div></div>', unsafe_allow_html=True)
    if sent and user_text.strip():
        handle_user_text(user_text.strip())

st.markdown('</div>', unsafe_allow_html=True)  # /chat-wrap


Overwriting app.py


In [24]:
# Reset & Run Streamlit + Ngrok (robust)
!pip -q install streamlit pyngrok requests

import os, time, subprocess, requests
from pyngrok import ngrok, conf

# =========================
# 0) ใส่ ngrok token ของคุณ (แนะนำเก็บใน env var)
# =========================
NGROK_TOKEN = "XXXXXXXXXXXXXXXXXXXXX"  # <-- แทนที่ตรงนี้ด้วยโทเคนใหม่ของคุณ
os.environ["NGROK_AUTHTOKEN"] = NGROK_TOKEN
conf.get_default().auth_token = os.environ["NGROK_AUTHTOKEN"]

# =========================
# 1) เคลียร์โปรเซสเก่า ๆ กันพอร์ตชน
# =========================
# kill streamlit ที่ค้าง (ถ้ามี)
!pkill -f "streamlit run app.py" || true

# kill ngrok เก่าที่ค้าง (ถ้ามี)
try:
    ngrok.kill()
except:
    pass

# =========================
# 2) รัน Streamlit แบบ background + เก็บ log
# =========================
log_path = "/tmp/streamlit.log"
streamlit_cmd = [
    "streamlit", "run", "app.py",
    "--server.port", "8501",
    "--server.address", "0.0.0.0",
    "--browser.gatherUsageStats", "false",
    "--server.headless", "true",
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
]
streamlit_proc = subprocess.Popen(
    streamlit_cmd,
    stdout=open(log_path, "w"),
    stderr=subprocess.STDOUT
)

# =========================
# 3) รอให้ Streamlit ติดเครื่อง (health check)
# =========================
def wait_ready(timeout=60):
    url = "http://localhost:8501/_stcore/health"
    start = time.time()
    while time.time() - start < timeout:
        try:
            r = requests.get(url, timeout=2)
            if r.ok:
                return True
        except Exception:
            pass
        time.sleep(1)
    return False

ok = wait_ready(60)

# =========================
# 4) เปิด ngrok tunnel
# =========================
public_tunnel = ngrok.connect(8501, "http")  # bind_tls ค่า default = True อยู่แล้ว
print("✅ Streamlit app is running!")
print("🌐 Open here:", public_tunnel.public_url)

# ถ้า health check ไม่ผ่าน ให้โชว์ท้าย log มาช่วยดีบัก
if not ok:
    print("\n⚠️ Streamlit health check failed; showing last 100 log lines:")
    try:
        with open(log_path, "r") as f:
            lines = f.read().splitlines()
        print("\n".join(lines[-100:]))
    except Exception as e:
        print("อ่าน log ไม่ได้:", e)



^C
✅ Streamlit app is running!
🌐 Open here: https://36c0d06758b7.ngrok-free.app
